In [188]:
import pandas as pd
import math
import numpy as np
import itertools

In [189]:
cb_wr = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/cb_wr.csv')
cb_wr['event'].iloc[:20]

0     pass_forward
1     pass_forward
2     pass_forward
3     pass_forward
4     pass_arrived
5     pass_arrived
6     pass_arrived
7     pass_arrived
8     pass_forward
9     pass_forward
10    pass_forward
11    pass_forward
12    pass_forward
13    pass_forward
14    pass_arrived
15    pass_arrived
16    pass_arrived
17    pass_arrived
18    pass_arrived
19    pass_arrived
Name: event, dtype: object

In [3]:
for idx, val in enumerate(cb_wr['event']):
    x = 0
    if x == 1:
        cb_wr['event'].iloc[idx] = 'between'
    if i == 'pass_forward':
        x = 1
    if i == 'pass_arrived':
        x = 0
    

NameError: name 'i' is not defined

In [190]:
#importing targeted receivers and cb_wr dataset from dylan

targetedReceiver = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/targetedReceiver.csv')
cb_wr = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/cb_wr.csv')

mask = ((cb_wr['position'] == 'CB') | (cb_wr['position'] == 'WR')) & ((cb_wr['event'] == 'pass_forward') | (cb_wr['event'] == 'pass_arrived'))
#getting only the pass forward and arrived events for CB and WR
cb_wr_passes = cb_wr[mask]

#only one game
game1 = cb_wr_passes[(cb_wr_passes['gameId'] == 2018090901)]
game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == 2018090901)]


#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x) 

In [171]:
games = cb_wr_passes['gameId'].unique()

playerEucDiff = pd.DataFrame()

for game in games:
    

    game1 = cb_wr_passes[(cb_wr_passes['gameId'] == game)]
    game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

    game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == game)]

    game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
    #replacing NaN with None
    game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

    #getting rid of instances where there is a WR that isn't the targeted one
    game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                     ['targetNflId'])) | \
                         (game_receiver_merge['position'] == 'CB'))]

    #zipping x and y
    game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

    #dropping x and y columns
    game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)


    #unique play ids
    plays = pd.Series(game_receiver_merge['playId'].unique())

    #creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
    merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

    for play in plays:
        df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
        unique_pos = df_temp['position'].unique()
        if len(unique_pos) == 2:
            merge_fixed = merge_fixed.append(df_temp)


    merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])

    #keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
    merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]

    #Calculating the euclidean distance between the players

    def euc_dist(item):
        return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                         ((item['x,y_x'][1] - item['x,y_y'][1])**2))

    merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)

        #cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
    merged2_fixed = merged2_fixed[(merged2_fixed['position_x'] == 'WR')]


    mins = []
    #This is dropping farther CBs. Will prob need to fix, bc it doesn't account for the case of 3 CBs and 1 WR. Would 
    # only drop the 1 furthest, not both furthest. 
    for play in plays:
        df_temp = merged2_fixed[((merged2_fixed['playId_x'] == play)) & ((merged2_fixed['event'] == 'pass_forward'))]

        if len(df_temp) > 1:

            idmin = df_temp[(df_temp['event'] == 'pass_forward')]['Euc_dist'].idxmin()
            nflmin = df_temp['nflId_y'].loc[idmin]
            min_ids = df_temp[df_temp['nflId_y'] == nflmin].index.tolist()

            mins.append(min_ids)



    flattened_mins = list(itertools.chain(*mins))

    merged2_fixed = merged2_fixed.loc[flattened_mins]

    #self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
    ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
    print(ForArrStack['time_Xagain'].head(10), ForArrStack['time_Yagain'].head(10), '\n\n\n\n')
    #Get rid of rows where the forward time is equal to or greater than the arrived time
    ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]

    #cut down on useless rows
    ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)

    #Calc the Euclidean Difference from pass forward and pass arrived
    ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']

    temp = pd.DataFrame()
    #created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
    temp['gameId'] = ForArrStack['gameId_x_Xagain']
    temp['playId'] = ForArrStack['playId_y_Xagain']
    temp['nflId_CB'] = ForArrStack['nflId_y']
    temp['startEucDist'] = ForArrStack['Euc_dist_Xagain']
    temp['Euc_diff'] = ForArrStack['Euc_diff']
    temp['Pct_change'] = (round(ForArrStack['Euc_diff'] / ForArrStack['Euc_dist_Xagain'], 4)) * 100

    playerEucDiff = playerEucDiff.append(temp, ignore_index=True)

0    2018-09-07T01:07:18.099Z
1    2018-09-07T01:09:12.700Z
2    2018-09-07T01:10:38.400Z
3    2018-09-07T01:16:54.900Z
4    2018-09-07T01:21:50.000Z
5    2018-09-07T01:36:52.799Z
6    2018-09-07T01:41:55.099Z
7    2018-09-07T01:47:28.099Z
8    2018-09-07T01:48:50.000Z
9    2018-09-07T01:49:28.700Z
Name: time_Xagain, dtype: object 0    2018-09-07T01:07:18.099Z
1    2018-09-07T01:09:12.700Z
2    2018-09-07T01:10:38.400Z
3    2018-09-07T01:16:54.900Z
4    2018-09-07T01:21:50.000Z
5    2018-09-07T01:36:52.799Z
6    2018-09-07T01:41:55.099Z
7    2018-09-07T01:47:28.099Z
8    2018-09-07T01:48:50.000Z
9    2018-09-07T01:49:28.700Z
Name: time_Yagain, dtype: object 




0    2018-09-09T17:03:24.000Z
1    2018-09-09T17:05:14.700Z
2    2018-09-09T17:11:30.000Z
3    2018-09-09T17:13:17.400Z
4    2018-09-09T17:19:02.299Z
5    2018-09-09T17:19:37.799Z
6    2018-09-09T17:20:17.000Z
7    2018-09-09T17:22:57.299Z
8    2018-09-09T17:29:48.299Z
9    2018-09-09T17:30:10.400Z
Name: time_Xagain, dtype: obj

0    2018-09-14T00:23:27.900Z
1    2018-09-14T00:26:56.599Z
2    2018-09-14T00:27:46.299Z
3    2018-09-14T00:32:21.099Z
4    2018-09-14T00:33:26.799Z
5    2018-09-14T00:39:55.700Z
6    2018-09-14T00:45:32.299Z
7    2018-09-14T00:47:55.599Z
8    2018-09-14T00:53:47.900Z
9    2018-09-14T00:58:01.200Z
Name: time_Xagain, dtype: object 0    2018-09-14T00:23:27.900Z
1    2018-09-14T00:26:56.599Z
2    2018-09-14T00:27:46.299Z
3    2018-09-14T00:32:21.099Z
4    2018-09-14T00:33:26.799Z
5    2018-09-14T00:39:55.700Z
6    2018-09-14T00:45:32.299Z
7    2018-09-14T00:47:55.599Z
8    2018-09-14T00:53:47.900Z
9    2018-09-14T00:58:01.200Z
Name: time_Yagain, dtype: object 




0    2018-09-16T17:03:07.500Z
1    2018-09-16T17:06:06.799Z
2    2018-09-16T17:07:37.599Z
3    2018-09-16T17:18:20.200Z
4    2018-09-16T17:24:52.400Z
5    2018-09-16T17:29:54.400Z
6    2018-09-16T17:31:47.900Z
7    2018-09-16T17:32:29.000Z
8    2018-09-16T17:46:37.299Z
9    2018-09-16T17:48:35.099Z
Name: time_Xagain, dtype: obj

KeyboardInterrupt: 

In [154]:
len(playerEucDiff)

0

In [191]:
#Merging game and targeted receivers
game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
#replacing NaN with None
game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

#getting rid of instances where there is a WR that isn't the targeted one
game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                 ['targetNflId'])) | \
                     (game_receiver_merge['position'] == 'CB'))]

#zipping x and y
game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

#dropping x and y columns
game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)

#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.4f' % x) 

game_receiver_merge.head()


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
4,2018-09-09T17:03:58.700Z,72,pass_forward,2543637.0000,CB,home,None,None,"(43.39, 46.13)"
5,2018-09-09T17:03:58.900Z,72,pass_arrived,2543637.0000,CB,home,None,None,"(44.4, 46.74)"
6,2018-09-09T17:03:58.700Z,72,pass_forward,2552389.0000,CB,home,None,None,"(57.67, 31.52)"
7,2018-09-09T17:03:58.900Z,72,pass_arrived,2552389.0000,CB,home,None,None,"(57.75, 32.12)"
8,2018-09-09T17:03:58.700Z,72,pass_forward,2556475.0000,CB,home,None,None,"(42.41, 13.79)"


In [192]:
# FIX SO WHEN ITERATING THRU EVERY GAME

#unique play ids
plays = pd.Series(game_receiver_merge['playId'].unique())

#creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

for play in plays:
    df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
    unique_pos = df_temp['position'].unique()
    if len(unique_pos) == 2:
        merge_fixed = merge_fixed.append(df_temp)
        
merge_fixed        


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
14,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
16,2018-09-09T17:04:43.799Z,96,pass_forward,2543637.0000,CB,home,None,None,"(44.52, 38.21)"
17,2018-09-09T17:04:43.799Z,96,pass_forward,2552389.0000,CB,home,None,None,"(60.82, 24.86)"
18,2018-09-09T17:04:43.799Z,96,pass_forward,2556475.0000,CB,home,None,None,"(44.34, 20.04)"
20,2018-09-09T17:04:43.799Z,96,pass_forward,2560916.0000,CB,home,None,None,"(44.77, 9.93)"
41,2018-09-09T17:10:23.799Z,291,pass_forward,496733.0000,CB,away,None,None,"(52.38, 42.44)"
42,2018-09-09T17:10:24.500Z,291,pass_arrived,496733.0000,CB,away,None,None,"(48.59, 41.34)"
43,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)"
44,2018-09-09T17:10:24.500Z,291,pass_arrived,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(52.82, 19.17)"
45,2018-09-09T17:10:23.799Z,291,pass_forward,2555344.0000,CB,away,None,None,"(49.43, 9.2)"


In [244]:
#merging on itself
merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])
merged2_fixed.head()


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
0,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
1,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2543637.0000,CB,home,None,None,"(44.52, 38.21)"
2,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2552389.0000,CB,home,None,None,"(60.82, 24.86)"
3,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2556475.0000,CB,home,None,None,"(44.34, 20.04)"
4,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2560916.0000,CB,home,None,None,"(44.77, 9.93)"


In [245]:
#keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]
      
    
merged2_fixed.iloc[:10]


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
1,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2543637.0000,CB,home,None,None,"(44.52, 38.21)"
2,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2552389.0000,CB,home,None,None,"(60.82, 24.86)"
3,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2556475.0000,CB,home,None,None,"(44.34, 20.04)"
4,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2560916.0000,CB,home,None,None,"(44.77, 9.93)"
5,2018-09-09T17:04:43.799Z,96,pass_forward,2543637.0000,CB,home,None,None,"(44.52, 38.21)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
10,2018-09-09T17:04:43.799Z,96,pass_forward,2552389.0000,CB,home,None,None,"(60.82, 24.86)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
15,2018-09-09T17:04:43.799Z,96,pass_forward,2556475.0000,CB,home,None,None,"(44.34, 20.04)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
20,2018-09-09T17:04:43.799Z,96,pass_forward,2560916.0000,CB,home,None,None,"(44.77, 9.93)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)"
26,2018-09-09T17:10:23.799Z,291,pass_forward,496733.0000,CB,away,None,None,"(52.38, 42.44)",291,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)"
29,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,496733.0000,CB,away,None,None,"(52.38, 42.44)"


In [246]:
#Calculating the euclidean distance between the players
    
def euc_dist(item):
    return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                     ((item['x,y_x'][1] - item['x,y_y'][1])**2))

merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)
    
merged2_fixed.head(25)


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2543637.0000,CB,home,None,None,"(44.52, 38.21)",17.0770
2,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2552389.0000,CB,home,None,None,"(60.82, 24.86)",17.6521
3,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2556475.0000,CB,home,None,None,"(44.34, 20.04)",1.3648
4,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2560916.0000,CB,home,None,None,"(44.77, 9.93)",11.2950
5,2018-09-09T17:04:43.799Z,96,pass_forward,2543637.0000,CB,home,None,None,"(44.52, 38.21)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",17.0770
10,2018-09-09T17:04:43.799Z,96,pass_forward,2552389.0000,CB,home,None,None,"(60.82, 24.86)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",17.6521
15,2018-09-09T17:04:43.799Z,96,pass_forward,2556475.0000,CB,home,None,None,"(44.34, 20.04)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",1.3648
20,2018-09-09T17:04:43.799Z,96,pass_forward,2560916.0000,CB,home,None,None,"(44.77, 9.93)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",11.2950
26,2018-09-09T17:10:23.799Z,291,pass_forward,496733.0000,CB,away,None,None,"(52.38, 42.44)",291,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",25.7784
29,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,496733.0000,CB,away,None,None,"(52.38, 42.44)",25.7784


In [247]:
merged2_fixed.head(10)

,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2543637.0000,CB,home,None,None,"(44.52, 38.21)",17.0770
2,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2552389.0000,CB,home,None,None,"(60.82, 24.86)",17.6521
3,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2556475.0000,CB,home,None,None,"(44.34, 20.04)",1.3648
4,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2560916.0000,CB,home,None,None,"(44.77, 9.93)",11.2950
5,2018-09-09T17:04:43.799Z,96,pass_forward,2543637.0000,CB,home,None,None,"(44.52, 38.21)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",17.0770
10,2018-09-09T17:04:43.799Z,96,pass_forward,2552389.0000,CB,home,None,None,"(60.82, 24.86)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",17.6521
15,2018-09-09T17:04:43.799Z,96,pass_forward,2556475.0000,CB,home,None,None,"(44.34, 20.04)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",1.3648
20,2018-09-09T17:04:43.799Z,96,pass_forward,2560916.0000,CB,home,None,None,"(44.77, 9.93)",96,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",11.2950
26,2018-09-09T17:10:23.799Z,291,pass_forward,496733.0000,CB,away,None,None,"(52.38, 42.44)",291,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",25.7784
29,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,496733.0000,CB,away,None,None,"(52.38, 42.44)",25.7784


In [248]:
#cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] == 'WR'))]
merged2_fixed.head(15)


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2543637.0000,CB,home,None,None,"(44.52, 38.21)",17.0770
2,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2552389.0000,CB,home,None,None,"(60.82, 24.86)",17.6521
3,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2556475.0000,CB,home,None,None,"(44.34, 20.04)",1.3648
4,2018-09-09T17:04:43.799Z,96,pass_forward,2508061.0000,WR,away,2018090901.0000,2508061.0000,"(43.56, 21.16)",96,2560916.0000,CB,home,None,None,"(44.77, 9.93)",11.2950
29,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,496733.0000,CB,away,None,None,"(52.38, 42.44)",25.7784
31,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,2555344.0000,CB,away,None,None,"(49.43, 9.2)",9.3850
32,2018-09-09T17:10:23.799Z,291,pass_forward,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(54.95, 16.79)",291,2556559.0000,CB,away,None,None,"(54.05, 16.77)",0.9002
45,2018-09-09T17:10:24.500Z,291,pass_arrived,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(52.82, 19.17)",291,496733.0000,CB,away,None,None,"(48.59, 41.34)",22.5699
47,2018-09-09T17:10:24.500Z,291,pass_arrived,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(52.82, 19.17)",291,2555344.0000,CB,away,None,None,"(47.89, 7.89)",12.3103
48,2018-09-09T17:10:24.500Z,291,pass_arrived,2543488.0000,WR,home,2018090901.0000,2543488.0000,"(52.82, 19.17)",291,2556559.0000,CB,away,None,None,"(52.73, 18.7)",0.4785


# This part doesn't work. The df_temp is messing up with the mask.

In [249]:
mins = []
#This is dropping farther CBs. Will prob need to fix, bc it doesn't account for the case of 3 CBs and 1 WR. Would 
# only drop the 1 furthest, not both furthest. 
for play in plays:
    mask = ((merged2_fixed['playId_x'] == play) & (merged2_fixed['event'] == 'pass_forward')) | ((merged2_fixed['playId_x'] == play) & (merged2_fixed['event'] == 'pass_arrived'))
    df_temp = merged2_fixed[mask]

    if len(df_temp) > 1:

        idmin = df_temp[(df_temp['event'] == 'pass_forward')]['Euc_dist'].idxmin()
        print(idmin)
        nflmin = df_temp['nflId_y'].loc[idmin]
        min_ids = df_temp[df_temp['nflId_y'] == nflmin].index.tolist()

        mins.append(min_ids)


        

In [250]:
flattened_mins = list(itertools.chain(*mins))

merged2_fixed = merged2_fixed.loc[flattened_mins]

In [251]:
merged2_fixed.head(10)

,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist


In [216]:
#self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
ForArrStack.columns


Index(['time_Xagain', 'playId_x', 'event_Xagain', 'nflId_x',
       'position_x_Xagain', 'team_x_Xagain', 'gameId_x_Xagain',
       'targetNflId_x_Xagain', 'x,y_x_Xagain', 'playId_y_Xagain', 'nflId_y',
       'position_y_Xagain', 'team_y_Xagain', 'gameId_y_Xagain',
       'targetNflId_y_Xagain', 'x,y_y_Xagain', 'Euc_dist_Xagain',
       'time_Yagain', 'event_Yagain', 'position_x_Yagain', 'team_x_Yagain',
       'gameId_x_Yagain', 'targetNflId_x_Yagain', 'x,y_x_Yagain',
       'playId_y_Yagain', 'position_y_Yagain', 'team_y_Yagain',
       'gameId_y_Yagain', 'targetNflId_y_Yagain', 'x,y_y_Yagain',
       'Euc_dist_Yagain'],
      dtype='object')

In [201]:
#Get rid of rows where the forward time is equal to or greater than the arrived time
ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]

In [202]:
#cut down on useless rows

ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)
print(ForArrStack.shape)

(0, 22)


In [203]:
#Calc the Euclidean Difference from pass forward and pass arrived
ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']
ForArrStack.columns

Index(['time_Xagain', 'playId_x', 'event_Xagain', 'nflId_x',
       'position_x_Xagain', 'team_x_Xagain', 'gameId_x_Xagain',
       'targetNflId_x_Xagain', 'x,y_x_Xagain', 'playId_y_Xagain', 'nflId_y',
       'position_y_Xagain', 'team_y_Xagain', 'targetNflId_y_Xagain',
       'x,y_y_Xagain', 'Euc_dist_Xagain', 'event_Yagain', 'team_x_Yagain',
       'x,y_x_Yagain', 'team_y_Yagain', 'x,y_y_Yagain', 'Euc_dist_Yagain',
       'Euc_diff'],
      dtype='object')

In [119]:
#created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
playerEucDiff = pd.DataFrame()
playerEucDiff['gameId'] = ForArrStack['gameId_x_Xagain']
playerEucDiff['playId'] = ForArrStack['playId_y_Xagain']
playerEucDiff['nflId_CB'] = ForArrStack['nflId_y']
playerEucDiff['startEucDist'] = ForArrStack['Euc_dist_Xagain']
playerEucDiff['Euc_diff'] = ForArrStack['Euc_diff']
playerEucDiff['Pct_change'] = (round(ForArrStack['Euc_diff'] / ForArrStack['Euc_dist_Xagain'], 4)) * 100
playerEucDiff

,gameId,playId,nflId_CB,startEucDist,Euc_diff,Pct_change
1,2018090600.0000,75,2555383.0000,4.1600,-2.5929,-62.3300
5,2018090600.0000,146,2555383.0000,4.6876,2.7931,59.5900
9,2018090600.0000,190,2557958.0000,5.7738,-1.7294,-29.9500
13,2018090600.0000,320,2539653.0000,3.9890,2.5711,64.4500
17,2018090600.0000,402,2552689.0000,3.5604,1.4577,40.9400
21,2018090600.0000,752,2539653.0000,3.7499,1.5530,41.4100
26,2018090600.0000,992,2539334.0000,17.9915,-1.5347,-8.5300
30,2018090600.0000,1037,2539653.0000,0.9617,0.7157,74.4200
34,2018090600.0000,1061,2539653.0000,0.6845,0.1298,18.9600
38,2018090600.0000,1085,2556445.0000,5.0019,-0.4282,-8.5600


In [ ]:
# Gotta iterate through each game. Also, note on the CB, WR pairing code. Last, need to do these as percentages, so
# is not skewed by scale of the coverage distances